In [1]:
import cobra
from cobra import Reaction
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import ast

# Load the MATLAB constructed cell line models and update IDs and add subsystems

In [3]:
models = ["LTED_FPKM_MATLAB.xml","MCF7_FPKM_MATLAB.xml",
         "MCF7_T_FPKM_MATLAB.xml","MCF7_F_FPKM_MATLAB.xml"]

rxn_ss = pd.read_csv('../subsystem_data/Recon3_rxn_subsystem.csv',index_col=0).set_index('rxn ID')


for filename in models:
    M = cobra.io.read_sbml_model(filename)

    ##################################
    # fix reaction and metabolite IDs
    ##################################
    for rxn in M.reactions:
        rxn.id = rxn.id.replace('__91__','_').replace('__93__','').replace('__45__','-')

    M.repair()

    for m in M.metabolites:
        m.id = m.id.replace('__91__','_').replace('__93__','')

    M.repair()

    ##################################
    # add ATPM reaction
    ##################################
    ATPM = Reaction('ATPM')
    M.add_reaction(ATPM)
    M.reactions.ATPM.lower_bound = 0
    M.reactions.ATPM.add_metabolites({'atp_c':-1.0,'adp_c':1.0,'pi_c':1.0})
    M.repair()

    ##################################
    # Set objective to biomass_reaction
    ##################################
    M.reactions.biomass_maintenance.objective_coefficient = 0
    M.reactions.biomass_reaction.objective_coefficient = 1
    
    ##################################
    # add subsystem info
    ##################################
    for rxn in rxn_ss.index:
        rxn_updated_id = rxn.replace('[','_').replace(']','')

        ss = ast.literal_eval(rxn_ss.loc[rxn]['subSystem'])

        M.reactions.get_by_id(rxn_updated_id).subsystem = ss

    # save
    # strip the '_MATLAB.xml' out of the filename and replace with json
    filename_new = '_'.join(filename.split('_')[:-1]) + '.json'
    print(filename_new)
    
    cobra.io.save_json_model(M,filename_new)

LTED_FPKM.json
MCF7_FPKM.json
MCF7_T_FPKM.json
MCF7_F_FPKM.json


In [21]:
for filename in ["LTED_FPKM.json", "MCF7_FPKM.json", "MCF7_T_FPKM.json", "MCF7_F_FPKM.json"]:
    model = cobra.io.load_json_model(filename)
    
    print(model.reactions.GAPD.subsystem)
    print(model.reactions.G3PD1.bounds)
    
    sol = cobra.flux_analysis.pfba(model)
    print(sol.fluxes['biomass_reaction'], sol.fluxes['biomass_maintenance'])

['Glycolysis/gluconeogenesis']
(-1000.0, 1000.0)
0.0 0.050276511056267854
['Glycolysis/gluconeogenesis']
(-1000.0, 1000.0)
0.0 0.08686654074673478
['Glycolysis/gluconeogenesis']
(-1000.0, 1000.0)
0.0 0.05718860802928058
['Glycolysis/gluconeogenesis']
(-1000.0, 1000.0)
0.0 0.06210063689001531
